In [ ]:
import xarray as xr
import plotdata as pd

In [ ]:
# pull data from files
try:
    ds = xr.open_dataset('nc/master')
    ds_fixed = ds.isel(model=[1, 2, 3, 5, 7, 8, 9, 12])
    ds_plev = xr.open_dataset('nc/master_plev')
    ds_plev_fixed = ds_plev.isel(model=[1, 2, 3, 5, 7, 8, 9, 12])
except FileNotFoundError:
    !python loaddata.py

In [ ]:
# get zonal time mean for variable
tm = ds_fixed.mean(dim=('time','lon'), skipna=True)

# operations on variables
w = (tm['rlut'] - tm['rldt']) + (tm['rsut'] - tm['rsdt'])
u = tm['FLNS'] - tm['FSNS'] + tm['hfss'] + tm['hfls'] + (tm['snow'] * 3.337e5)

# calculate time mean anomalies
z  = w.sel(exp='AquaControl')

# plot lat-lon time mean
pd.plot_allmodels(z - y, box=False)

In [ ]:
# get time mean for variable
tm_plev = ds_plev_fixed.mean(dim='month', skipna=True).sel(plev=92500)

# operations on variables
p = tm_plev['ua']
q = tm_plev['va']

# calculate time mean anomalies
u = p.sel(exp='LandControl') - p.sel(exp='AquaControl')
v = q.sel(exp='LandControl') - q.sel(exp='AquaControl')

# plot quivers
pd.quivermap_allexperiments(u, v)

In [ ]:
import matplotlib.pyplot as plt

w = ds['pr']

# operations on variables
zm    = w.mean(dim='lon')
land  = w.sel(lon=slice(0, 45)).mean(dim='lon')
ocean = xr.concat([w.sel(lon=slice(-180, 0)),
                   w.sel(lon=slice(45, 180))], dim='lon').mean(dim='lon')

# calculate time mean anomalies
z  = ocean.sel(exp='LandControl') - ocean.sel(exp='AquaControl')

# plot Hovmoller diagrams
pd.plot_allexperiments(z, '$W/m^2$', x='month', box=True)

In [ ]:
# get time mean for variable
tm = ds_fixed.mean(dim='month', skipna=True).median(dim='model').

# calculate time mean anomalies
z  = tm.sel(exp='LandControl') - tm.sel(exp='AquaControl')

pd.plot_allvariables(z, box=False, levels=14)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

tm = ds_fixed.sel(lat=slice(-30,30),lon=slice(0,45)).mean(dim=('month','lat','lon'))

p = (tm['rlus'] - tm['rlds']) + (tm['rsus'] - tm['rsds']) + (tm['hfls'] + tm['hfss'])
q = tm['ts']

# calculate time mean anomalies
x = p.sel(exp='LandControl')
y = q.sel(exp='LandControl')

fig, ax = plt.subplots()
for i, model in list(enumerate(x.model.values)):
    ax.scatter(x[i], y[i], label=model)

ax.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid(True)

linregress(x,y)[2]**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

c_north = [-6.8650e-6, 1.6743e-3, -1.4162e-1, 4.9755, -50.7586]
c_south = [-1.7732e-5, -3.4685e-3, -2.3494e-1, -6.4824, -56.6094]

p_north = np.poly1d(c_north)
p_south = np.poly1d(c_south)


x = ds.lat.values
y = np.piecewise(x, [x <= 0, x >= 0], [lambda x: p_south(x), lambda x: p_north(x)])

fig, ax = plt.subplots()
ax.plot(x,y)